In [2]:
import torch
from torch import nn

In [2]:
x = torch.randn(100, 3)
layer = nn.Linear(3, 5)
print(layer(x).shape)
print(layer.weight)
print(layer.bias)

torch.Size([100, 5])
Parameter containing:
tensor([[-0.1108, -0.4514, -0.4163],
        [ 0.3495,  0.0874,  0.4593],
        [ 0.1155,  0.4322, -0.3844],
        [-0.1602,  0.2817, -0.5329],
        [-0.3002,  0.3061, -0.4402]], requires_grad=True)
Parameter containing:
tensor([-0.3681, -0.1281,  0.4130, -0.5634, -0.4491], requires_grad=True)


In [6]:
x = torch.randn(2, 5)
layer = nn.ReLU() # 음수는 0 양수는 linear
print(x)
print(layer(x))

tensor([[ 0.2882,  1.3782,  0.4333,  0.0688,  0.8333],
        [-1.2900,  0.2007, -1.1845, -0.6450, -1.0957]])
tensor([[0.2882, 1.3782, 0.4333, 0.0688, 0.8333],
        [0.0000, 0.2007, 0.0000, 0.0000, 0.0000]])


In [9]:
x = torch.randn(3, 7) # 어떤 노드를 통과된 상태라 하자. 3개, 7 feature
drop = nn.Dropout(p = 0.3) # p는 죽일 확률!! 논문에선 살릴 확률이였다.
print(drop(x)) # 각 개가 통과마다 다르게 drop 된다..!

tensor([[ 0.3459, -1.5823,  2.9730,  1.2473, -0.0000, -1.0688,  1.3578],
        [-0.6621,  0.0000,  2.5391,  3.8966, -0.0255, -0.6990, -0.0000],
        [-0.3712, -0.1636, -0.0000, -1.2739,  0.7860,  1.0813,  0.5819]])


In [2]:
# dropout은 정말 train, test 다르게 작동하나??

class sample_model(nn.Module):
    def __init__(self):
        super().__init__()
        self.drop_layer = nn.Sequential(nn.Linear(5, 7),
                                        nn.Dropout(p = 0.3))
    def forward(self, x):
        x = self.drop_layer(x)
        return x

model = sample_model()
model.train()
x = torch.randn(3, 5)
print(model(x)) # 0.7의 확률로 죽임

model.eval()
print(model(x)) # 다 살려 그 대신 각 노드마다 0.7 곱

tensor([[-2.0236, -0.8471, -0.0000, -1.6398, -0.0000, -0.1545,  0.0000],
        [-0.0000,  0.3171, -0.2573, -0.1815,  0.5417, -0.1272, -0.2058],
        [-0.0119,  0.0000, -0.8282, -0.2105, -0.0000, -0.1014,  0.0000]],
       grad_fn=<MulBackward0>)
tensor([[-1.4165, -0.5930, -1.1434, -1.1478, -1.6408, -0.1081,  0.9608],
        [-0.1661,  0.2220, -0.1801, -0.1270,  0.3792, -0.0891, -0.1441],
        [-0.0084,  0.1061, -0.5797, -0.1474, -0.3469, -0.0710,  0.2844]],
       grad_fn=<AddmmBackward0>)


In [9]:
layer = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=3, stride=1, padding=1) # stride =1, padding =0이 디폴트
'''
in_channel == 필터의 채널 수
out_chanel == 필터의 수 즉, 출력되는 데이터의 채널의 수
'''
print(layer(torch.randn(32, 1, 5, 5)).shape) # 개체행렬!
# nn.linear(3, 5) # 채 채 -> 이놈은 채 또는 개체를 원함 -> 즉, 개 x 3이야지 통과 가능하다.
# nn.Conv2d(3, 5) # 채 채 -> 애는 채 행 열 또는 개 채 행 열을 원함, 개 x 3 x 행 x 열

torch.Size([32, 2, 5, 5])


In [12]:
layer = nn.Conv2d(3, 5, 3, stride =2 , padding =1)
print(layer(torch.randn(32, 3, 5, 5)).shape)
print(layer.weight.shape) # 개 채 행 렬 -> 3 x 3 filter가 3의 channel을 가지고 총 5개가 있다.

torch.Size([32, 5, 3, 3])
torch.Size([5, 3, 3, 3])


In [15]:
conv1 = nn.Conv2d(1, 8, 6, stride=2)
x = torch.randn(32, 1, 28, 28)
print(conv1(x).shape)

conv2 = nn.Conv2d(8, 16, 3, padding=1)
print(conv2(conv1(x)).shape)

Maxpool = nn.MaxPool2d(kernel_size=2, stride=(2, 2))
print(Maxpool(conv2(conv1(x))).shape)

torch.Size([32, 8, 12, 12])
torch.Size([32, 16, 12, 12])
torch.Size([32, 16, 6, 6])


In [18]:
maxpool = nn.MaxPool2d(2) # 2 로만 줘도 자동 kernel_size = 2, stride = (2, 2)
x = torch.randn(1, 1, 6, 6)
print(x)
print(maxpool(x))

tensor([[[[ 0.3499, -0.3534,  0.4921, -0.0284, -0.2667, -1.0244],
          [ 0.9035,  1.5047,  0.2959,  1.3930,  0.5262, -0.0326],
          [ 1.1675, -0.7535,  0.1152, -1.1356, -0.4297,  0.2672],
          [-1.2772,  0.9734,  0.3927, -0.1595,  0.4006,  0.2541],
          [ 1.3374, -1.6176, -0.1437, -0.5672,  0.1777,  0.6217],
          [-0.6175,  1.3684,  2.3634,  0.0053,  0.9664, -1.2419]]]])
tensor([[[[1.5047, 1.3930, 0.5262],
          [1.1675, 0.3927, 0.4006],
          [1.3684, 2.3634, 0.9664]]]])


In [19]:
avgpool = nn.AvgPool2d(2)
x = torch.randn(1, 1, 6, 6)
print(x)
print(avgpool(x))

tensor([[[[ 0.7793,  0.1524,  0.2603, -0.7056,  0.6494,  0.7938],
          [ 1.0123, -0.2509, -1.0570, -0.3321, -0.7396,  0.2015],
          [-1.6956,  0.3449, -0.4866, -0.8647, -0.6779,  1.0431],
          [-0.1455, -1.5248,  0.3526, -1.1165, -1.4569,  1.8551],
          [ 0.6834, -0.1479,  1.4771,  0.0593,  0.8920, -0.5520],
          [-0.6460,  1.0205, -1.6543,  0.0640,  0.6712,  0.5071]]]])
tensor([[[[ 0.4233, -0.4586,  0.2263],
          [-0.7552, -0.5288,  0.1909],
          [ 0.2275, -0.0135,  0.3796]]]])


In [21]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(1, 8, 6, stride=2)
        self.conv2 = nn.Conv2d(8, 16, 3, padding=1)
        self.Maxpool2 = nn.MaxPool2d(2)
        self.fc = nn.Linear(16 * 6 * 6, 10) # 잘모르겠으면 Flatten 이건까지만 하고 실행

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.Maxpool2(x)
        x = torch.flatten(x, start_dim=1)
        x = self.fc(x)
        return x

x = torch.randn(32, 1, 28, 28)
model = CNN()
print(model(x).shape)

torch.Size([32, 10])


## .parameters() vs .modules() vs .children() 그리고 isinstance의 활용

In [21]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Sequential(nn.Linear(2,3),
                                 nn.ReLU())
        self.fc2 = nn.Sequential(nn.Linear(3,4),
                                 nn.ReLU())
        self.fc_out = nn.Sequential(nn.Linear(4,1), # 이진분류
                                    nn.Sigmoid())
    def forward(self,x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc_out(x)
        return x

model = MLP()
print(model(torch.randn(2,2)).shape)
print(model)

torch.Size([2, 1])
MLP(
  (fc1): Sequential(
    (0): Linear(in_features=2, out_features=3, bias=True)
    (1): ReLU()
  )
  (fc2): Sequential(
    (0): Linear(in_features=3, out_features=4, bias=True)
    (1): ReLU()
  )
  (fc_out): Sequential(
    (0): Linear(in_features=4, out_features=1, bias=True)
    (1): Sigmoid()
  )
)


In [7]:
'''
[layer0 weight 값, layer0 bias 값, layer1 weight 값, layer1 bias 값, ...]
'''
list(model.parameters())

[Parameter containing:
 tensor([[ 0.4411,  0.5475],
         [-0.2305,  0.5315],
         [-0.3816,  0.0659]], requires_grad=True),
 Parameter containing:
 tensor([-0.2984, -0.2058,  0.4123], requires_grad=True),
 Parameter containing:
 tensor([[ 0.2673,  0.3347, -0.1339],
         [ 0.2274,  0.4008, -0.2736],
         [-0.4424, -0.0530, -0.3074],
         [ 0.1328, -0.2152,  0.2612]], requires_grad=True),
 Parameter containing:
 tensor([0.3346, 0.2851, 0.2692, 0.4073], requires_grad=True),
 Parameter containing:
 tensor([[-0.4130,  0.4999,  0.4465,  0.2102]], requires_grad=True),
 Parameter containing:
 tensor([0.4567], requires_grad=True)]

In [22]:
model = MLP()

In [25]:
'''
For transfer learning
<10개 분류 문제를 5개로...>
'''
model = MLP()
print([p for p in model.parameters() if p.requires_grad])
print("=" * 50)
for p in model.parameters(): # 전체 Freeze
    p.requires_grad = False

model.fc_out = nn.Linear(4, 10) # 출력만 체인지!

params = [p for p in model.parameters() if p.requires_grad]
print(params)

from torch import optim

'''
이렇게 하면 기존 모델의 가중치는 그대로 두고
마지막 fc_out 레이어만 새로운 데이터에 맞게 학습!
'''
optimizer = optim.Adam(params, lr=0.1) # 마지막만 가중치 학습 ㄱㄱ하자 기존껀 그대로 쓰고

[Parameter containing:
tensor([[ 0.2376, -0.3245],
        [ 0.3080,  0.0575],
        [ 0.6979,  0.2428]], requires_grad=True), Parameter containing:
tensor([-0.0224, -0.1967,  0.1500], requires_grad=True), Parameter containing:
tensor([[ 0.3464,  0.2828,  0.3086],
        [-0.0090, -0.4900,  0.1332],
        [-0.1833, -0.2232, -0.1966],
        [ 0.1517,  0.1053, -0.1945]], requires_grad=True), Parameter containing:
tensor([ 0.1392, -0.3340, -0.0250, -0.1186], requires_grad=True), Parameter containing:
tensor([[-0.3074,  0.4248,  0.4775, -0.0182]], requires_grad=True), Parameter containing:
tensor([0.2389], requires_grad=True)]
[Parameter containing:
tensor([[-0.2784,  0.2752, -0.0354,  0.3425],
        [ 0.4612, -0.4997, -0.2331,  0.2940],
        [ 0.0696, -0.4541,  0.4058, -0.3344],
        [-0.3250,  0.3717, -0.3814, -0.3557],
        [-0.4980, -0.0508, -0.1319, -0.0313],
        [ 0.2423, -0.3420, -0.4910, -0.0967],
        [-0.2382, -0.4480,  0.3457, -0.0204],
        [ 0.0296,

In [12]:
'''
[('layer0.weight', weight 값), ('layer0.bias', bias 값), ('layer1.weight', weight 값), ('layer1.bias', bias 값), ...]
'''
list(model.named_parameters())

[('fc1.0.weight',
  Parameter containing:
  tensor([[-0.3308, -0.0155],
          [-0.1607,  0.4018],
          [-0.1613, -0.0032]])),
 ('fc1.0.bias',
  Parameter containing:
  tensor([-0.6536,  0.5953, -0.5930])),
 ('fc2.0.weight',
  Parameter containing:
  tensor([[-0.2898,  0.0487,  0.4600],
          [ 0.5438,  0.4170, -0.0354],
          [ 0.5531, -0.1187, -0.5636],
          [-0.2376,  0.5725, -0.2811]])),
 ('fc2.0.bias',
  Parameter containing:
  tensor([-0.2515,  0.2962,  0.3924, -0.3640])),
 ('fc_out.weight',
  Parameter containing:
  tensor([[ 0.1011, -0.0024,  0.2741, -0.3737],
          [-0.2407,  0.3261, -0.2067,  0.1658],
          [-0.3740, -0.3387,  0.3064, -0.1720],
          [-0.1757, -0.4754, -0.2350, -0.2415],
          [-0.2969, -0.2228,  0.2518, -0.3152],
          [-0.3919,  0.2561, -0.4509, -0.0173],
          [-0.1330, -0.4636, -0.2654,  0.0723],
          [-0.0270, -0.2983,  0.1694, -0.2709],
          [-0.4674,  0.2276,  0.4860,  0.4239],
          [ 0.0619, 

In [13]:
for name, p in model.named_parameters():
    print(name)
    print(p)

fc1.0.weight
Parameter containing:
tensor([[-0.3308, -0.0155],
        [-0.1607,  0.4018],
        [-0.1613, -0.0032]])
fc1.0.bias
Parameter containing:
tensor([-0.6536,  0.5953, -0.5930])
fc2.0.weight
Parameter containing:
tensor([[-0.2898,  0.0487,  0.4600],
        [ 0.5438,  0.4170, -0.0354],
        [ 0.5531, -0.1187, -0.5636],
        [-0.2376,  0.5725, -0.2811]])
fc2.0.bias
Parameter containing:
tensor([-0.2515,  0.2962,  0.3924, -0.3640])
fc_out.weight
Parameter containing:
tensor([[ 0.1011, -0.0024,  0.2741, -0.3737],
        [-0.2407,  0.3261, -0.2067,  0.1658],
        [-0.3740, -0.3387,  0.3064, -0.1720],
        [-0.1757, -0.4754, -0.2350, -0.2415],
        [-0.2969, -0.2228,  0.2518, -0.3152],
        [-0.3919,  0.2561, -0.4509, -0.0173],
        [-0.1330, -0.4636, -0.2654,  0.0723],
        [-0.0270, -0.2983,  0.1694, -0.2709],
        [-0.4674,  0.2276,  0.4860,  0.4239],
        [ 0.0619, -0.1196,  0.2553,  0.3430]], requires_grad=True)
fc_out.bias
Parameter containing

In [14]:
list(model.modules())

[MLP(
   (fc1): Sequential(
     (0): Linear(in_features=2, out_features=3, bias=True)
     (1): ReLU()
   )
   (fc2): Sequential(
     (0): Linear(in_features=3, out_features=4, bias=True)
     (1): ReLU()
   )
   (fc_out): Linear(in_features=4, out_features=10, bias=True)
 ),
 Sequential(
   (0): Linear(in_features=2, out_features=3, bias=True)
   (1): ReLU()
 ),
 Linear(in_features=2, out_features=3, bias=True),
 ReLU(),
 Sequential(
   (0): Linear(in_features=3, out_features=4, bias=True)
   (1): ReLU()
 ),
 Linear(in_features=3, out_features=4, bias=True),
 ReLU(),
 Linear(in_features=4, out_features=10, bias=True)]

In [16]:
'''
Checking for gradient vanishing
'''
print([m for m in model.modules() if isinstance(m,nn.Linear)])
print("=" * 50)
print([m.weight for m in model.modules() if isinstance(m,nn.Linear)])
print("=" * 50)
print([m.weight.grad for m in model.modules() if isinstance(m,nn.Linear)])

[Linear(in_features=2, out_features=3, bias=True), Linear(in_features=3, out_features=4, bias=True), Linear(in_features=4, out_features=10, bias=True)]
[Parameter containing:
tensor([[-0.3308, -0.0155],
        [-0.1607,  0.4018],
        [-0.1613, -0.0032]]), Parameter containing:
tensor([[-0.2898,  0.0487,  0.4600],
        [ 0.5438,  0.4170, -0.0354],
        [ 0.5531, -0.1187, -0.5636],
        [-0.2376,  0.5725, -0.2811]]), Parameter containing:
tensor([[ 0.1011, -0.0024,  0.2741, -0.3737],
        [-0.2407,  0.3261, -0.2067,  0.1658],
        [-0.3740, -0.3387,  0.3064, -0.1720],
        [-0.1757, -0.4754, -0.2350, -0.2415],
        [-0.2969, -0.2228,  0.2518, -0.3152],
        [-0.3919,  0.2561, -0.4509, -0.0173],
        [-0.1330, -0.4636, -0.2654,  0.0723],
        [-0.0270, -0.2983,  0.1694, -0.2709],
        [-0.4674,  0.2276,  0.4860,  0.4239],
        [ 0.0619, -0.1196,  0.2553,  0.3430]], requires_grad=True)]
[None, None, None]


In [23]:
'''
weight initialization에 활용
'''
for m in model.modules():
    if isinstance(m, nn.Linear):
        #nn.init.kaiming_normal_(m.weight)
        nn.init.constant_(m.weight, 1)

print([m.weight for m in model.modules() if isinstance(m, nn.Linear)])
print("=" * 50)
print([p for p in model.parameters() if p.requires_grad])

[Parameter containing:
tensor([[1., 1.],
        [1., 1.],
        [1., 1.]], requires_grad=True), Parameter containing:
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], requires_grad=True), Parameter containing:
tensor([[1., 1., 1., 1.]], requires_grad=True)]
[Parameter containing:
tensor([[1., 1.],
        [1., 1.],
        [1., 1.]], requires_grad=True), Parameter containing:
tensor([ 0.6548, -0.2828, -0.1646], requires_grad=True), Parameter containing:
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], requires_grad=True), Parameter containing:
tensor([ 0.1869,  0.2428, -0.0914, -0.4958], requires_grad=True), Parameter containing:
tensor([[1., 1., 1., 1.]], requires_grad=True), Parameter containing:
tensor([-0.2523], requires_grad=True)]


In [26]:
list(model.children())

[Sequential(
   (0): Linear(in_features=2, out_features=3, bias=True)
   (1): ReLU()
 ),
 Sequential(
   (0): Linear(in_features=3, out_features=4, bias=True)
   (1): ReLU()
 ),
 Linear(in_features=4, out_features=10, bias=True)]

In [27]:
x = torch.randn(2, 2)
list(model.children())[0](x)

tensor([[0.0000, 0.0000, 0.6766],
        [0.0000, 0.0000, 0.0000]])

In [28]:
print(*list(model.children())[:2])
sub_network = nn.Sequential(*list(model.children())[:2])
print(sub_network)
print(sub_network(x))

Sequential(
  (0): Linear(in_features=2, out_features=3, bias=True)
  (1): ReLU()
) Sequential(
  (0): Linear(in_features=3, out_features=4, bias=True)
  (1): ReLU()
)
Sequential(
  (0): Sequential(
    (0): Linear(in_features=2, out_features=3, bias=True)
    (1): ReLU()
  )
  (1): Sequential(
    (0): Linear(in_features=3, out_features=4, bias=True)
    (1): ReLU()
  )
)
tensor([[0.3480, 0.0000, 0.0000, 0.0000],
        [0.1392, 0.0000, 0.0000, 0.0000]])


## ModuleList vs Sequential

In [30]:
fc = nn.Linear(3, 3)
layer_list = [fc for _ in range(5)]
layers1 = nn.Sequential(*layer_list)
layers2 = nn.ModuleList(layer_list) # * 쓰고 지랄 엠병 할 필요 없노
print(layers1)
print("=" * 50)
print(layers2)

Sequential(
  (0): Linear(in_features=3, out_features=3, bias=True)
  (1): Linear(in_features=3, out_features=3, bias=True)
  (2): Linear(in_features=3, out_features=3, bias=True)
  (3): Linear(in_features=3, out_features=3, bias=True)
  (4): Linear(in_features=3, out_features=3, bias=True)
)
ModuleList(
  (0-4): 5 x Linear(in_features=3, out_features=3, bias=True)
)


In [31]:
x=torch.randn(1,3)
print(layers1(x))

# print(layers2(x)) # error! -> Seq에는 Forward()가 정의되어있어서 그냥 가능..
for layer in layers2:
    x = layer(x)
print(x)

tensor([[-0.5715,  0.3017, -0.3979]], grad_fn=<AddmmBackward0>)
tensor([[-0.5715,  0.3017, -0.3979]], grad_fn=<AddmmBackward0>)


In [43]:
'''
<걍 리스트 쓰지 왜 nn.ModuleList 를 쓸까?>
'''
class testNet(nn.Module):
    def __init__(self):
        super().__init__()

        #self.Module_List = [nn.Linear(3,3), nn.Linear(3,3)] # -> 등록 못함..
        self.Module_List = nn.ModuleList([nn.Linear(3,3), nn.Linear(3,3)])

    def forward(self,x):
        for layer in self.Module_List:
            x = layer(x)
        return x

model=testNet()
print(model(torch.randn(1,3)))

#print(model) # 그냥 리스트로 하면 등록이 안돼있다!
print(list(model.parameters()))
#optimizer = optim.Adam(model.parameters(), lr = 0.1) # 등록이 안돼있으면 parameter를 못 찾는다!

tensor([[-0.7920, -0.2759, -0.4472]], grad_fn=<AddmmBackward0>)
[Parameter containing:
tensor([[-0.0513, -0.3149, -0.5710],
        [-0.4224,  0.2163, -0.4706],
        [-0.0809, -0.3872,  0.1710]], requires_grad=True), Parameter containing:
tensor([-0.4993, -0.4372,  0.2502], requires_grad=True), Parameter containing:
tensor([[-0.0152,  0.4593, -0.5006],
        [-0.2672, -0.4765, -0.1522],
        [ 0.5358,  0.0264, -0.1710]], requires_grad=True), Parameter containing:
tensor([-0.5164, -0.5323, -0.3616], requires_grad=True)]


In [49]:
'''
<그럼 nn.Sequential 쓰고 말지 왜 굳이 nn.ModuleList?>
'''
class small_block(nn.Module):
    def __init__(self):
        super().__init__()
        self.block_x = nn.Linear(1,1)
        self.block_y = nn.Linear(1,1)

    def forward(self, x, y):
        x = self.block_x(x)
        y = self.block_y(y)
        return x, y

block = small_block()
print(block)
print("=" * 50)
model = nn.Sequential(block, block)
print(model)
print("=" * 50)
# model(torch.randn(1), torch.randn(1)) # error!
# nn.Sequential 이 가지고 있는 forward 함수를 call 하기 때문에 입력을 두 개 넣으면 안된다!!
# nn.Seq의 forward(self, x) 이런식으로 하나만 받을수 ㅇㅆ노

model = nn.ModuleList([block,block])
x = torch.randn(1)
y = torch.randn(1)
for block in model: # -> forward() 정의
    x, y = block(x,y)
print(x, y)

small_block(
  (block_x): Linear(in_features=1, out_features=1, bias=True)
  (block_y): Linear(in_features=1, out_features=1, bias=True)
)
Sequential(
  (0): small_block(
    (block_x): Linear(in_features=1, out_features=1, bias=True)
    (block_y): Linear(in_features=1, out_features=1, bias=True)
  )
  (1): small_block(
    (block_x): Linear(in_features=1, out_features=1, bias=True)
    (block_y): Linear(in_features=1, out_features=1, bias=True)
  )
)
tensor([-0.3195], grad_fn=<ViewBackward0>) tensor([1.3596], grad_fn=<ViewBackward0>)
